In [ ]:
from PIL import Image
import os
import warnings

def verify_images(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            with warnings.catch_warnings():
                warnings.simplefilter('error', Image.DecompressionBombWarning)
                img = Image.open(file_path)  # Try to open the image file
                img.verify()  # Verify that it is an image file
        except (IOError, SyntaxError, Image.DecompressionBombWarning) as e:
            print(f"Removing invalid image: {file_path}")
            os.remove(file_path)  # Remove the invalid image file

# Verify images in the Cat and Dog directories
verify_images('D:/45-Days-Challege-JS-Python/PetImages/Cat')
verify_images('D:/45-Days-Challege-JS-Python/PetImages/Dog')

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

# Define paths
train_dir = 'D:/45-Days-Challege-JS-Python/PetImages'

# Load the VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification for cats vs dogs

# Create the new model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Data augmentation and preprocessing for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,       # Normalize pixel values to [0, 1]
    shear_range=0.2,      # Randomly apply shearing transformations
    zoom_range=0.2,       # Randomly zoom in on images
    horizontal_flip=True  # Randomly flip images horizontally
)

# Data augmentation and preprocessing for the validation set
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess training dataset
train_generator = train_datagen.flow_from_directory(
    os.path.join(train_dir, 'train'),
    target_size=(224, 224),  # Resize images to 224x224 pixels
    batch_size=32,
    class_mode='binary'      # Binary classification
)

# Load and preprocess validation dataset
validation_generator = test_datagen.flow_from_directory(
    os.path.join(train_dir, 'validation'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'PetImage'

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy*100:.2f}%')

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
